In [2]:
import sys
sys.path = ["../../../.."] + sys.path # 切换到项目目录下

import scanpy as sc
import scvelo as scv
import velovgi

from ray import tune, air
from ray.air import session

Global seed set to 0
/home/huang/.conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/home/huang/.conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


## 1. 重新组织目标函数后的流程

1. 目标函数，数据读取过程就在目标函数里写了
    - 之前写到外面的话，每次创建一个Trail，除了显式地传入config参数，额外都会把现有的全局变量复制一遍，。
    - 现在数据读取写在里面，包也尽可能在里面导入，每次创建一个Trail传入的数据量就少得多了。

In [33]:
from pytorch_lightning import loggers
from torch_geometric import seed_everything

def train_velovgi(config):
    # 包导入
    # 这里需要指定目录，否则找不到
    import sys
    sys.path = ["/home/huang/PyCode/scRNA/Other/velovgi_workstation"] + sys.path

    import scvelo as scv
    import velovgi

    # 数据读取与基本设置
    adata = scv.read("/home/huang/PyCode/scRNA/data/Gastrulation/erythroid_lineage.h5ad")
    cluster_key = "celltype"
    batch_key="stage"
    batch_pair_list = [
        ["E7.0", "E7.25"],
        ["E7.25", "E7.5"],
        ["E7.5", "E7.75"],
        ["E7.75", "E8.0"],
        ["E8.0", "E8.25"],
        ["E8.25", "E8.5"],
    ]
    cluster_edges = [
    ("Blood progenitors 1", "Blood progenitors 2"), 
    ("Blood progenitors 2", "Erythroid1"), 
    ("Erythroid1", "Erythroid2"), 
    ("Erythroid2", "Erythroid3")
    ] # 已知的细胞类型间的分化信息

    ################################################################################
    # 邻居数量的两个参数
    n_bnn_neighbors = config["n_bnn_neighbors"]
    n_knn_neighbors = config["n_knn_neighbors"]
    
    knn_mask, bnn_mask, subsample_adata = velovgi.pp.preprocess(adata, n_bnn_neighbors, n_knn_neighbors, sample_mode="random", batch_key=batch_key, batch_pair_list=batch_pair_list)

    name = ""
    for k,v in config.items():
        name += "%s_%s,"%(k, v)
    name = name[:-1]

    seed_everything(0)
    # 模型训练
    logger = loggers.TensorBoardLogger(save_dir="./log", name=name)
    velovgi.tl.VELOVGI.setup_anndata(adata=subsample_adata, spliced_layer="Ms", unspliced_layer="Mu")
    velovgi_model = velovgi.tl.VELOVGI(subsample_adata)
    velovgi_model.train(logger=logger, max_epochs=50)

    # 模型恢复
    velovgi.tl.add_velovi_outputs_to_adata(subsample_adata, velovgi_model) # 模型输出
    velovgi.pp.moment_recover(adata, subsample_adata) # 恢复

    # 速率计算
    scv.tl.velocity_graph(adata)
    scv.pl.velocity_embedding(adata, color=cluster_key)
    scv.pl.velocity_embedding_stream(adata, color=cluster_key, save="%s.png"%name)

    # 计算指标评价
    adata_velo = velovgi.tl.pre_metric(adata)
    exp_metrics = velovgi.tl.summary_metric(adata_velo, cluster_edges, cluster_key)[-1] # 计算指标汇总后的结果

    session.report({"CBDir": exp_metrics["CBDir"], "ICVCoh": exp_metrics["ICVCoh"]})


2. 搜索空间，网格搜索

In [31]:
search_space = {
    "n_bnn_neighbors": tune.grid_search([15, 30, 50]),
    "n_knn_neighbors": tune.grid_search([15, 30, 50]),
}

3. 执行调参

In [32]:
from ray.tune.schedulers import ASHAScheduler

name = "test_experiment"

tuner = tune.Tuner(
    train_velovgi,
    tune_config=tune.TuneConfig(
        metric="CBDir",
        mode="max",
        scheduler=ASHAScheduler()
    ),
    run_config=air.RunConfig(
        local_dir="./results", # Trail内部具体输出结果在这里保存
        name=name # 开启调参的Tensorboard日志
    ),
    param_space=search_space,
)

results = tuner.fit()

2023-06-04 17:39:10,364	WARNING callback.py:142 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(train_velovgi pid=182395) Global seed set to 0
(train_velovgi pid=182395) /home/huang/.conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
(train_velovgi pid=182395)   new_rank_zero_deprecation(
(train_velovgi pid=182395) /home/huang/.conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
(train_velovgi pid=182395)   return new_rank_zero_deprecation(*args, **kwargs)


(train_velovgi pid=182395) Filtered out 47456 genes that are detected 20 counts (shared).
(train_velovgi pid=179922) Filtered out 47456 genes that are detected 20 counts (shared).
(train_velovgi pid=179922) Filtered out 47456 genes that are detected 20 counts (shared).
(train_velovgi pid=179922) Filtered out 47456 genes that are detected 20 counts (shared).
(train_velovgi pid=179922) Filtered out 47456 genes that are detected 20 counts (shared).
(train_velovgi pid=179922)     finished (0:00:00) --> added [repeated 2x across cluster]
(train_velovgi pid=179922) Filtered out 47456 genes that are detected 20 counts (shared).
(train_velovgi pid=179922) Filtered out 47456 genes that are detected 20 counts (shared).
(train_velovgi pid=179922) Filtered out 47456 genes that are detected 20 counts (shared).
(train_velovgi pid=179922) Figure(640x480) [repeated 2x across cluster]
(train_velovgi pid=179922) Filtered out 47456 genes that are detected 20 counts (shared).
(train_velovgi pid=182395) No

(train_velovgi pid=182395) OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
(train_velovgi pid=182450) OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
(train_velovgi pid=182450) OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
(train_velovgi pid=182450) OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
(train_velovgi pid=182450) OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
(train_velovgi pid=182450) OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


(train_velovgi pid=182395) smoothing...
(train_velovgi pid=182395) WARNING: The neighbor graph has an unexpected format (e.g. computed outside scvelo) 
(train_velovgi pid=182395) or is corrupted (e.g. due to subsetting). Consider recomputing with `pp.neighbors`.
(train_velovgi pid=182395) computing moments based on connectivities
(train_velovgi pid=182450) computing moments based on connectivities
(train_velovgi pid=182450) computing moments based on connectivities
(train_velovgi pid=182450) computing moments based on connectivities
(train_velovgi pid=182450) computing moments based on connectivities
(train_velovgi pid=182450) computing moments based on connectivities
(train_velovgi pid=182450) computing moments based on connectivities
(train_velovgi pid=182395)     finished (0:00:00) --> added 
(train_velovgi pid=182395)     'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


(train_velovgi pid=182395) GPU available: False, used: False
(train_velovgi pid=182395) TPU available: False, using: 0 TPU cores
(train_velovgi pid=182395) IPU available: False, using: 0 IPUs
(train_velovgi pid=182395) HPU available: False, using: 0 HPUs
(train_velovgi pid=182395) Missing logger folder: ./log/n_bnn_neighbors_15,n_knn_neighbors_15
(train_velovgi pid=182450) Missing logger folder: ./log/n_bnn_neighbors_15,n_knn_neighbors_15


(train_velovgi pid=182395) 初始训练，初始化runner参数
(train_velovgi pid=182395) choosing neighbor minibatch
Epoch 50/50:  98%|█████████▊| 49/50 [07:54<00:09,  9.99s/it, loss=4.5e+05, v_num=0] [repeated 2x across cluster]


(train_velovgi pid=182395) `Trainer.fit` stopped: `max_epochs=50` reached.
(train_velovgi pid=182450) `Trainer.fit` stopped: `max_epochs=50` reached.
(train_velovgi pid=182450) `Trainer.fit` stopped: `max_epochs=50` reached.
(train_velovgi pid=182450) `Trainer.fit` stopped: `max_epochs=50` reached.
(train_velovgi pid=182450) `Trainer.fit` stopped: `max_epochs=50` reached.
(train_velovgi pid=182450) `Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 50/50: 100%|██████████| 50/50 [08:04<00:00,  9.68s/it, loss=4.44e+05, v_num=0] [repeated 2x across cluster]
(train_velovgi pid=182395) WARNING: The neighbor graph has an unexpected format (e.g. computed outside scvelo) 
(train_velovgi pid=182395) or is corrupted (e.g. due to subsetting). Consider recomputing with `pp.neighbors`.
(train_velovgi pid=182450) or is corrupted (e.g. due to subsetting). Consider recomputing with `pp.neighbors`.
(train_velovgi pid=182395) computing velocity graph (using 1/24 cores)
(train_velovgi pid=182395)   0%|          | 0/9815 [00:00<?, ?cells/s]
(train_velovgi pid=182450) or is corrupted (e.g. due to subsetting). Consider recomputing with `pp.neighbors`.
(train_velovgi pid=182450) or is corrupted (e.g. due to subsetting). Consider recomputing with `pp.neighbors`.
(train_velovgi pid=182450)   0%|          | 0/9815 [00:00<?, ?cells/s]
(train_velovgi pid=182395)     finished (0:00:20) --> added 
(train_velovgi pid=182395)     'velocity_graph', sparse 

2023-06-04 17:52:08,570	INFO tune.py:945 -- Total run time: 778.22 seconds (778.20 seconds for the tuning loop).


4. 查看训练日志与最优结果: tensorboard --logdir

In [35]:
results.get_best_result().config

{'n_bnn_neighbors': 45, 'n_knn_neighbors': 15}